In [5]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import requests
import time
from datetime import date
import calendar
#Download grequests by typing 'pip install grequest' into your terminal 
import grequests
import time
from datetime import date
import calendar
from config import key
import json

In [6]:
#Declare data base path 
from sqlalchemy import create_engine
database_path = "music.sqlite"

In [7]:
#Create an engine connection 
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

# Music csv file

In [10]:
#read in music data csv: you can download the csv from the following link 
#https://www.kaggle.com/edumucelli/spotifys-worldwide-daily-song-ranking
#be sure to change the music_file directory to reflect where you saved the downloaded csv 
music_file = "../music_data.csv"
music_data_df = pd.read_csv(music_file)

music_data_df.head()

,Position,Track Name,Artist,Streams,URL,Date,Region
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,1/1/2017,EC
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,1/1/2017,EC
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,https://open.spotify.com/track/3QwBODjSEzelZyV...,1/1/2017,EC
3,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/1/2017,EC
4,5,Safari,J Balvin,14269,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,1/1/2017,EC


In [12]:
#Clean up the data by renaming the columns 
music_data_df = music_data_df.rename(columns = {'Region':'country_codes'})
music_data_df['country_codes'] = music_data_df['country_codes'].str.upper()
music_data_df.head()

,Position,Track Name,Artist,Streams,URL,Date,country_codes
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,1/1/2017,EC
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,1/1/2017,EC
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,https://open.spotify.com/track/3QwBODjSEzelZyV...,1/1/2017,EC
3,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/1/2017,EC
4,5,Safari,J Balvin,14269,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,1/1/2017,EC


# Country codes web scraping

In [20]:
#Plug in the url that we will be scraping
url = 'https://www.worldatlas.com/aatlas/ctycodes.htm'

In [21]:
#create our table variable 
tables = pd.read_html(url)

In [22]:
#create our data frame 
counrty_df = tables[0]
counrty_df.columns = ['COUNTRY', 'A2 (ISO)', 'A3 (UN)', 'NUM (UN)', 'DIALING CODE']
counrty_df.head()

,COUNTRY,A2 (ISO),A3 (UN),NUM (UN),DIALING CODE
0,COUNTRY,A2 (ISO),A3 (UN),NUM (UN),DIALING CODE
1,Afghanistan,AF,AFG,4,93
2,Albania,AL,ALB,8,355
3,Algeria,DZ,DZA,12,213
4,American Samoa,AS,ASM,16,1-684


In [23]:
counrty_df = counrty_df.iloc[1:]
counrty_df = counrty_df.rename(columns = {"COUNTRY":"Country", "A2 (ISO)":"country_codes", "A3 (UN)":"A3 (UN)", "NUM (UN)":
                                          "NUM (UN)", "DIALING CODE":"DIALING CODE" })
#drop the columns we don't need 
counrty_df= counrty_df.drop(columns=["A3 (UN)", "NUM (UN)", "DIALING CODE"])

counrty_df.head()

,Country,country_codes
1,Afghanistan,AF
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS
5,Andorra,AD


In [24]:
# merge counrty_df to music_data_df with an inner join
music_df = music_data_df.merge(counrty_df, how="inner", on =["country_codes", "country_codes"])
music_df.head()

,Position,Track Name,Artist,Streams,URL,Date,country_codes,Country
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,1/1/2017,EC,Ecuador
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,1/1/2017,EC,Ecuador
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,https://open.spotify.com/track/3QwBODjSEzelZyV...,1/1/2017,EC,Ecuador
3,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/1/2017,EC,Ecuador
4,5,Safari,J Balvin,14269,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,1/1/2017,EC,Ecuador


# Gather Genre 

In [58]:
#tracks_df_unique = music_data_df[["Track Name", "Artist"]].copy()
#tracks_df_unique.drop_duplicates(inplace=True)
#tracks_df_unique.rename(columns={"Track Name":"Track"}, inplace=True)
#tracks_df_unique.head()

In [59]:
#Collect all the unique track names and unique artists 
tracks_df = music_data_df[['Track Name', 'Artist']].groupby(['Track Name', 'Artist']).nunique()
tracks_df = tracks_df[["Track Name","Artist"]].copy()
#Clean up the df so it only includes the columns we want 
tracks_df.rename(columns={"Track Name":"track_count","Artist":"art_count"}, inplace=True)
tracks_df = tracks_df.reset_index()
tracks_df.rename(columns={"Track Name":"Track"}, inplace=True)
tracks_df_unique = tracks_df[["Track","Artist"]].copy()
tracks_df_unique.head()

,Track,Artist
0,"""All That Is or Ever Was or Ever Will Be""",Alan Silvestri
1,"""Read All About It, Pt. III""",Emeli Sandé
2,#99,JVG
3,#Askip,Black M
4,#Biziz - feat. Lil Bege,Reynmen


In [60]:
urls=[]
for index,row in tracks_df_unique.iterrows(): 
    track = row["Track"].replace("#","%23")
    artist = row["Artist"].replace("#","%23")
    url =  f"http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key={key}&artist={artist}&track={track}&format=json"
    urls.append(url)

In [61]:
# A list to hold our things to do via async
async_list = []

for u in urls:
    action_item = grequests.get(u, timeout=1000000)
    async_list.append(action_item)

response = grequests.map(async_list)

In [30]:
#remove all the info that is N/A 
response = [x for x in response if x is not None ]
len(response)

239

In [31]:
additional_info = []
row={}

for x in response:
    row ={}
    try:
        row["track"] = x.json()["track"]["name"]
        row["album"] = x.json()["track"]["album"]["title"]
        row["artist"] = x.json()["track"]["artist"]["name"]
        row["genre"] = x.json()["track"]["toptags"]["tag"][0]["name"]
        
        additional_info.append(row)
    except (IndexError,KeyError):
        a=1
additional_info = pd.DataFrame(additional_info)

In [32]:
#Rename the columns to match our music data frame 
additional_info.rename(columns = {"artist":"Artist", "genre":"Genre","track":"Track Name"}, inplace=True)


In [33]:
# merge the music_data_df with the additional info data frame 
music_df_final = music_df.merge(additional_info, how="inner", on =["Artist","Track Name"])
#music_df.head()


In [34]:
#check out what our data frame looks like  
music_df_final.head()

,Position,Track Name,Artist,Streams,URL,Date,country_codes,Country,album,Genre
0,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/1/2017,EC,Ecuador,Vente Pa' Ca,remix
1,5,Vente Pa' Ca,Ricky Martin,11590,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/2/2017,EC,Ecuador,Vente Pa' Ca,remix
2,5,Vente Pa' Ca,Ricky Martin,12280,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/3/2017,EC,Ecuador,Vente Pa' Ca,remix
3,4,Vente Pa' Ca,Ricky Martin,12413,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/4/2017,EC,Ecuador,Vente Pa' Ca,remix
4,4,Vente Pa' Ca,Ricky Martin,12337,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/5/2017,EC,Ecuador,Vente Pa' Ca,remix


In [35]:
#Count how many calls were completed 
len(music_df_final)

16794

In [36]:
#Print out the final results to make sure that we're getting what we're looking for 
music_df_final

,Position,Track Name,Artist,Streams,URL,Date,country_codes,Country,album,Genre
0,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/1/2017,EC,Ecuador,Vente Pa' Ca,remix
1,5,Vente Pa' Ca,Ricky Martin,11590,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/2/2017,EC,Ecuador,Vente Pa' Ca,remix
2,5,Vente Pa' Ca,Ricky Martin,12280,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/3/2017,EC,Ecuador,Vente Pa' Ca,remix
3,4,Vente Pa' Ca,Ricky Martin,12413,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/4/2017,EC,Ecuador,Vente Pa' Ca,remix
4,4,Vente Pa' Ca,Ricky Martin,12337,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/5/2017,EC,Ecuador,Vente Pa' Ca,remix
5,4,Vente Pa' Ca,Ricky Martin,12770,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/6/2017,EC,Ecuador,Vente Pa' Ca,remix
6,5,Vente Pa' Ca,Ricky Martin,14144,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/7/2017,EC,Ecuador,Vente Pa' Ca,remix
7,5,Vente Pa' Ca,Ricky Martin,12998,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/8/2017,EC,Ecuador,Vente Pa' Ca,remix
8,6,Vente Pa' Ca,Ricky Martin,10843,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/9/2017,EC,Ecuador,Vente Pa' Ca,remix
9,6,Vente Pa' Ca,Ricky Martin,10633,https://open.spotify.com/track/7DM4BPaS7uofFul...,1/10/2017,EC,Ecuador,Vente Pa' Ca,remix


In [37]:
#convert the results into a csv file 
music_df_final.to_csv("music_data_final.csv")

In [38]:
#import the results into sql 
music_df_final.to_sql('music_data', con=conn, if_exists='replace')